In [1]:
import pandas as pd
import numpy as np
import glob
import os
import datetime
import matplotlib.pyplot as plt
from functools import reduce


%matplotlib inline

In [158]:
FORECAST_PERIOD = 7

In [159]:
def load_data(files):
    
    # initiate dict to store data
    dfs = {}

    for f in files:

        # get name of the water body from the filename
        database = f.split('\\')[-1].replace('.csv','')

        # load the data
        df = pd.read_csv(f)

        # get start and end data of the data
        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'])

        # store data in dictionary by water body name
        dfs[database] = df
    
    return dfs

In [160]:
def get_lagged_value(df, targets, regex, replace_str, shortname, n, period=30, avg=True, difference=True, wrt='nmin1'):
    
    min_periods = int(period*0.66)
    
    if period == 30:
        pname = 'm'    # month
    elif period == 7:
        pname = 'w'    # week
    else:
        print('Error: period not valid')
        raise
    
    if avg:
        vname = 'avg'
    else:
        vname = 'sum'
    
    for col in df.filter(regex=regex):

        if col in targets:
            continue
            
        # location
        if replace_str != 'auto':
            location = col.replace(replace_str, '')[:4]
        else:
            location = col[-8:]
            
        # create summed features per period
        drop_cols = []
        for ni in range(1, n+1):
            
            # calculate the avg in period n
            col_n = '%s_%s_%d%s_%s'%(shortname, vname, ni, pname, location)
            if ni == 1:
                df[col_n] = df[col].rolling(window=period*ni, min_periods=min_periods).sum() 
            else:
                df[col_n] = df[col].rolling(window=period*ni, min_periods=min_periods).sum() - df[col].rolling(window=period*(ni-1), min_periods=min_periods).sum()
            
            if avg:
                df[col_n] = df[col_n]/period
            
            if difference:
                # calculate the difference with respect to the previous period
                
                if ni > 1:
                    
                    # define colnames
                    col_d = '%s_d%s_%d%s_%s'%(shortname, vname, ni, pname, location)
                    if wrt == 'nmin1':
                        col_nmin1 = '%s_%s_%d%s_%s'%(shortname, vname, ni-1, pname, location)
                    elif wrt == '1':
                        col_nmin1 = '%s_%s_%d%s_%s'%(shortname, vname, 1, pname, location)
                    else: 
                        print('Error: invalid choice for wrt', wrt)
                        raise

                    # calculate difference
                    df[col_d] = df[col_nmin1] - df[col_n]
                    
                    # drop column used to calculate difference
                    drop_cols.append(col_n)
        
        # drop column in case difference is used (if not used the list will be empty)
        #df.drop(columns=drop_cols, inplace=True)
                
    return df

def get_avg_features(df, cols_to_avg_list):
    
    for cols, colname in cols_to_avg_list:
        df[colname] = df[cols].mean(axis=1)
        
    return df

def get_window_features(df, targets):
    
    for target in targets:
        df['span2_%s'%target] = df['target'].ewm(span=2).mean()

def create_features(df, targets, nperiods, cols_to_avg_list, cols_to_drop_list, difference=True):
    
    # create month feature
    df['month'] = df['Date'].dt.month
    df = pd.get_dummies(df, columns=['month'], prefix='month')
    df.set_index('Date', inplace=True)
        
    # shift all columns apart from the target solumns with the FORECAST PERIOD
    for c in set(df.columns) - set(targets) - set([c for c in df.columns if 'month' in c]):    
        df[c] = df[c].shift(FORECAST_PERIOD)

    # create autocorrelation feature -- this is based on 1 datapoint so this is sensitive to statistical fluctuations!
    for c in targets:
        df['auto_%s'%c] = df[c].shift(FORECAST_PERIOD)

    # average columns with high correlation
    # replace any columns which is also a target with the respective 'auto_' column, to make sure to take the shifted data!
    for avg_list in cols_to_avg_list:
        avg_list[0] = ['auto_%s'%c if c in targets else c for c in avg_list[0]]
    # get average values
    df = get_avg_features(df, cols_to_avg_list)
    
    # drop columns we do not need anymore (for instance because combined in previous step)
    # note that we cannot throw away column directly in previous step because in some cases targets and features are
    # combined
    df = df.drop(columns=cols_to_drop_list)

    ### get lagged rainfall features -- weeks
    regex = '^avg_Rainfall'
    replace_str = 'avg_Rainfall_'
    shortname = 'ar'
    df = get_lagged_value(df, targets, regex, replace_str, shortname, nperiods, period=7, avg=False, difference=difference)

    ### get lagged features -- months (difference with n-1 value)
    cols = [['^Depth_to_Groundwater', 'Depth_to_Groundwater_', 'd'],
            ['^avg_Depth_to_Groundwater', 'avg_Depth_to_Groundwater_', 'ad'],
            ['^Temperature', 'Temperature_', 't'],
            ['^avg_Temperature', 'avg_Temperature_', 'at'],
            ['^Hydrometry', 'Hydrometry_', 'h'],
            ['^avg_Hydrometry', 'avg_Hydrometry_', 'ah'],
            ['^Volume', 'Volume_', 'v'],
            ['^avg_Volume', 'avg_Volume_', 'av'],
            #['^Flow_Rate', 'Flow_Rate_', 'f'],
            #['^auto', 'auto', 'a']
           ]

    for col in cols:
        df = get_lagged_value(df, targets, *col, nperiods, period=30, avg=True, difference=True, wrt='nmin1')

    ### get lagged features -- months (difference with current value)
    cols = [['^auto', 'auto', 'a']
           ]
    
    for col in cols:
        df = get_lagged_value(df, targets, *col, 6, period=30, avg=True, difference=False)

    
    print(df.shape)
    df = df.dropna()
        
    return df
        


In [161]:
datadir = '../data/'
cleandir = os.path.join(datadir, 'clean')
featsengdir = os.path.join(datadir, 'featseng')

In [162]:
files = glob.glob(os.path.join(cleandir, '*.csv'))
filename_cols = glob.glob(os.path.join(cleandir, '*.xlsx'))[0]

In [163]:
dfs = load_data(files)
names = list(dfs.keys())

dfcols = pd.read_excel(filename_cols, engine='openpyxl')

In [164]:
cols_to_avg = {
    'Aquifer_Auser': [
        [['Rainfall_Gallicano', 'Rainfall_Pontetetto',
         'Rainfall_Monte_Serra', 'Rainfall_Orentano', 'Rainfall_Borgo_a_Mozzano',
         'Rainfall_Piaggione', 'Rainfall_Calavorno', 'Rainfall_Croce_Arcana',
         'Rainfall_Tereglio_Coreglia_Antelminelli',
         'Rainfall_Fabbriche_di_Vallico'],
         'avg_Rainfall_area1'],
        [['Temperature_Orentano', 'Temperature_Monte_Serra',
         'Temperature_Lucca_Orto_Botanico'],
         'avg_Temperature_area1'],
        [['Depth_to_Groundwater_SAL', 'Depth_to_Groundwater_PAG',
         'Depth_to_Groundwater_COS', 'Depth_to_Groundwater_DIEC'],
         'avg_Depth_to_Groundwater_area1'],
        [['Hydrometry_Monte_S_Quirico', 'Hydrometry_Piaggione'],
         'avg_Hydrometry_area1'],
        [['Volume_POL', 'Volume_CC2'],
         'avg_Volume_POL_CC2_area1'],
        [['Volume_CSA', 'Volume_CSAL'],
         'avg_Volume_CSA_area1']
    ],
    'Aquifer_Petrignano' : [
        [['Temperature_Bastia_Umbra', 'Temperature_Petrignano'],
         'avg_Temperature_area1']
    ],
    'River_Arno': [
        [['Rainfall_Le_Croci', 'Rainfall_Cavallina', 'Rainfall_S_Agata', 
          'Rainfall_Mangona', 'Rainfall_S_Piero'],
        'avg_Rainfall_area1'],
    ],
    'Lake_Bilancino': [
        [['Rainfall_S_Piero', 'Rainfall_Mangona', 'Rainfall_S_Agata',
          'Rainfall_Cavallina', 'Rainfall_Le_Croci', 'Temperature_Le_Croci'],
         'avg_Rainfall_area1']]
}


cols_to_drop = {'Aquifer_Auser': ['Rainfall_Gallicano', 'Rainfall_Pontetetto',
                                  'Rainfall_Monte_Serra', 'Rainfall_Orentano', 'Rainfall_Borgo_a_Mozzano',
                                  'Rainfall_Piaggione', 'Rainfall_Calavorno',
                                  'Temperature_Orentano', 'Temperature_Monte_Serra',
                                  'Temperature_Lucca_Orto_Botanico',
                                  'Depth_to_Groundwater_PAG', 'Depth_to_Groundwater_DIEC',
                                  'Hydrometry_Monte_S_Quirico', 'Hydrometry_Piaggione',
                                  'Volume_POL', 'Volume_CC2',
                                  'Volume_CSA', 'Volume_CSAL',
                                 ],
                'Aquifer_Petrignano' : ['Temperature_Bastia_Umbra', 'Temperature_Petrignano'],
                'River_Arno': ['Rainfall_Le_Croci', 'Rainfall_Cavallina', 'Rainfall_S_Agata', 
                               'Rainfall_Mangona', 'Rainfall_S_Piero'],
                'Lake_Bilancino': ['Rainfall_S_Piero', 'Rainfall_Mangona', 'Rainfall_S_Agata',
                                   'Rainfall_Cavallina', 'Rainfall_Le_Croci', 'Temperature_Le_Croci']
               }



In [165]:
names

['Aquifer_Auser', 'Aquifer_Petrignano', 'Lake_Bilancino', 'River_Arno']

In [166]:
nperiods = 3
for name in names:
    df = dfs[name].copy()
    print(name, df.shape)
    targets = dfcols[(dfcols['name']==name) & (dfcols['coltype']=='target')]['colname'].values
    dffeats = create_features(df, targets, nperiods, cols_to_avg[name], cols_to_drop[name], difference=True)
    print(name, dffeats.shape)
    dffeats.to_csv(os.path.join(featsengdir, 'D%d'%FORECAST_PERIOD, '%s.csv'%(name)))
    

Aquifer_Auser (2008, 26)
(2008, 81)
Aquifer_Auser (1432, 81)
Aquifer_Petrignano (1643, 8)
(1643, 47)
Aquifer_Petrignano (1600, 47)
Lake_Bilancino (6026, 9)
(6026, 34)
Lake_Bilancino (6001, 34)
River_Arno (6026, 7)
(6026, 26)
River_Arno (5792, 26)
